In [32]:
import pandas as pd
import numpy as np
import geopandas as gpd
import salem
import json

In [33]:
Export = pd.read_csv("../1_Country_Emission/Export_woANTUSR.csv")
# We want to make use of Gross_output

In [34]:
Export

,Country,Industries,TID,TFD,FID,FFD,HID,HFD,Gross_output,Tol_export
0,AFG,Agriculture,1.538228e+06,2.982173e+06,1.118930e+06,488121.500819,419297.800230,2.494051e+06,4.520401e+06,1.607052e+06
1,AFG,Fishing,8.204556e+04,1.021952e+05,3.288965e+04,691.735172,49155.910904,1.015034e+05,1.842407e+05,3.358138e+04
2,AFG,Mining and Quarrying,2.571376e+05,2.229511e+04,4.083923e+04,701.700171,216298.329572,2.159341e+04,2.794327e+05,4.154093e+04
3,AFG,Food & Beverages,4.822587e+05,8.496689e+05,6.115531e+03,416.200579,476143.215230,8.492527e+05,1.331928e+06,6.531732e+03
4,AFG,Textiles and Wearing Apparel,1.526739e+05,1.494905e+05,2.357414e+04,2232.457909,129099.781240,1.472581e+05,3.021645e+05,2.580659e+04
...,...,...,...,...,...,...,...,...,...,...
4858,ZWE,"Education, Health and Other Services",4.053929e+05,9.166833e+05,4.051682e+05,10999.052329,224.685887,9.056842e+05,1.322076e+06,4.161672e+05
4859,ZWE,Private Households,5.454534e+04,3.839635e+05,5.453544e+04,2445.266074,9.902411,3.815182e+05,4.385088e+05,5.698070e+04
4860,ZWE,Others,1.878119e+05,3.851117e+05,1.878020e+05,7345.654559,9.980098,3.777660e+05,5.729236e+05,1.951476e+05
4861,ZWE,Re-export & Re-import,2.625942e+05,4.400673e+05,2.625817e+05,14370.062704,12.486401,4.256972e+05,7.026615e+05,2.769518e+05


## Question: What should we do with ROW?
#### $$\text{concentration factors} = \frac{\text{concentration caused by sector i in country j}}{\text{total output of sector i in country j}}$$
#### We only have one total output including all sectors for ROW.
#### We split the total output of ROW to different sectors according to the each sector's fraction among all sectors of other 189 countries.
#### $$\text{AGR output of ROW} = \frac{\text{AGR output of 189 countries}}{\text{total output of 189 countries}} \times \text{total output of ROW}$$

### Get rid of ANT: Netherlands Antilles (Netherlands) and USR: USSR

## Update: instead of getting rid of these two countries, using concentrations factors for the rest of the world

In [35]:
# Export = Export.drop(index=range(120*26,121*26),axis=0)
# Export = Export.drop(index=range(174*26,175*26),axis=0)
# # Reset column names and row indexes
# Export = Export.reset_index(drop=True)

In [36]:
Countries = Export['Country'].unique().tolist()

In [37]:
'SUD' in Countries

True

In [38]:
sector_name = Export['Industries'].unique().tolist()[:-1]

In [39]:
# Assign sectors in EORA to sectors in CEDS
sectors = ['AGR/RCOO','RCOO','IND','IND','IND','IND','IND','IND','IND','IND','IND','IND','ENE',
           'IND','RCOC','RCOC','RCOC','RCOC','NRTR/SHP','0','0','RCOC/RCOO','RCOC','RCOR','0','0']
agr = [0]

ene = [i for i, x in enumerate(sectors) if x == 'ENE']

ind = [i for i, x in enumerate(sectors) if x == 'IND']

nrtr = [sectors.index('NRTR/SHP')]

rcor = [i for i, x in enumerate(sectors) if x == 'RCOR']

rcoc = [i for i, x in enumerate(sectors) if x == 'RCOC']
rcoc.append(sectors.index('RCOC/RCOO'))

rcoo = [i for i, x in enumerate(sectors) if x == 'RCOO']
rcoo.append(sectors.index('RCOC/RCOO'))
rcoo.append(sectors.index('AGR/RCOO'))

shp = [sectors.index('NRTR/SHP')]

#  What belongs to IND or RCO also belongs to SLV
slv = list(np.unique(ind + rcor + rcoc + rcoo))

# Every sector will have waste.
wst = list(np.unique(slv + shp + nrtr + ene + agr))

A = ['agr','ene','ind','nrtr','rcor','rcoc','rcoo','shp','slv','wst']

In [40]:
sector_pos = {}
sector_pos['agr'] = [0]
sector_pos['ene'] = [i for i, x in enumerate(sectors) if x == 'ENE']

sector_pos['ind'] = [i for i, x in enumerate(sectors) if x == 'IND']

sector_pos['nrtr'] = [sectors.index('NRTR/SHP')]

sector_pos['rcor'] = [i for i, x in enumerate(sectors) if x == 'RCOR']

sector_pos['rcoc'] = [i for i, x in enumerate(sectors) if x == 'RCOC']
sector_pos['rcoc'].append(sectors.index('RCOC/RCOO'))

sector_pos['rcoo'] = [i for i, x in enumerate(sectors) if x == 'RCOO']
sector_pos['rcoo'].append(sectors.index('RCOC/RCOO'))
sector_pos['rcoo'].append(sectors.index('AGR/RCOO'))

sector_pos['shp'] = [sectors.index('NRTR/SHP')]

#  What belongs to IND or RCO also belongs to SLV
sector_pos['slv'] = list(np.unique(ind + rcor + rcoc + rcoo))

# Every sector will have waste.
sector_pos['wst'] = list(np.unique(slv + shp + nrtr + ene + agr))

In [41]:
sector_pos

{'agr': [0],
 'ene': [12],
 'ind': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13],
 'nrtr': [18],
 'rcor': [23],
 'rcoc': [14, 15, 16, 17, 22, 21],
 'rcoo': [1, 21, 0],
 'shp': [18],
 'slv': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 21, 22, 23],
 'wst': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  21,
  22,
  23]}

In [42]:
list(np.array(sector_name)[shp])

['Transport']

In [43]:
total_output_sector = {}
for i in Countries[:-1]:
    total_output_sector[i]={}
    for j in A:
        total_output_sector[i][j]= {}
total_output_sector

{'AFG': {'agr': {},
  'ene': {},
  'ind': {},
  'nrtr': {},
  'rcor': {},
  'rcoc': {},
  'rcoo': {},
  'shp': {},
  'slv': {},
  'wst': {}},
 'ALB': {'agr': {},
  'ene': {},
  'ind': {},
  'nrtr': {},
  'rcor': {},
  'rcoc': {},
  'rcoo': {},
  'shp': {},
  'slv': {},
  'wst': {}},
 'DZA': {'agr': {},
  'ene': {},
  'ind': {},
  'nrtr': {},
  'rcor': {},
  'rcoc': {},
  'rcoo': {},
  'shp': {},
  'slv': {},
  'wst': {}},
 'AND': {'agr': {},
  'ene': {},
  'ind': {},
  'nrtr': {},
  'rcor': {},
  'rcoc': {},
  'rcoo': {},
  'shp': {},
  'slv': {},
  'wst': {}},
 'AGO': {'agr': {},
  'ene': {},
  'ind': {},
  'nrtr': {},
  'rcor': {},
  'rcoc': {},
  'rcoo': {},
  'shp': {},
  'slv': {},
  'wst': {}},
 'ATG': {'agr': {},
  'ene': {},
  'ind': {},
  'nrtr': {},
  'rcor': {},
  'rcoc': {},
  'rcoo': {},
  'shp': {},
  'slv': {},
  'wst': {}},
 'ARG': {'agr': {},
  'ene': {},
  'ind': {},
  'nrtr': {},
  'rcor': {},
  'rcoc': {},
  'rcoo': {},
  'shp': {},
  'slv': {},
  'wst': {}},
 'ARM'

In [44]:
for i in range(len(Countries)):
    if Countries[i] != "ROW":
        for j in A:
            temp_index = [index + 26*i for index in sector_pos[j]]
            total_output_sector[Countries[i]][j] = Export.iloc[temp_index]['Gross_output'].sum()
#     else:
#         for j in A:
#             total_output_sector[Countries[i]][j] = 0

In [45]:
len(total_output_sector)

187

In [46]:
total_output = {}
for i in Countries:
    total_output[i]={}

In [47]:
# Summation of the value insde a dictionary
def dictsum(myDict):      
    sum = 0
    for i in myDict: 
        sum = sum + myDict[i]     
    return sum

In [48]:
for i in Countries[:-1]:
    total_output[i] = dictsum(total_output_sector[i])
total_output['ROW'] = Export[Export['Country'] == 'ROW']['Gross_output'].sum()
total_output_sector['ROW'] = total_output['ROW']

In [49]:
total_output

{'AFG': 96310465.26708463,
 'ALB': 63669264.348169334,
 'DZA': 705944528.5393372,
 'AND': 14355632.86984872,
 'AGO': 470776807.8337958,
 'ATG': 6316736.8837180985,
 'ARG': 2287696662.262965,
 'ARM': 46334617.378714785,
 'ABW': 14269258.836237019,
 'AUS': 5575519002.09327,
 'AUT': 1976931138.260456,
 'AZE': 200951969.87174383,
 'BHS': 43375675.199495405,
 'BHR': 179420436.23278946,
 'BGD': 966558140.1446431,
 'BRB': 21704623.230739158,
 'BLR': 87833163.32934877,
 'BEL': 2742172223.915438,
 'BLZ': 8398055.995796708,
 'BEN': 44738525.79199384,
 'BMU': 22203668.609052695,
 'BTN': 11656763.545881728,
 'BOL': 144451422.4494715,
 'BIH': 88167454.96886101,
 'BWA': 80275616.49835554,
 'BRA': 7657704336.71229,
 'VGB': 6313736.062434502,
 'BRN': 60426034.41050014,
 'BGR': 364620496.6832907,
 'BFA': 55282404.18513811,
 'BDI': 12550941.33634413,
 'KHM': 102216295.27232502,
 'CMR': 141830075.42761827,
 'CAN': 7911738662.010307,
 'CPV': 8544061.623409802,
 'CYM': 19566695.653979648,
 'CAF': 8664860.1

In [50]:
file=open("total_output.json",'w')
json.dump(total_output,file)
file.close()

In [51]:
# for i in total_output_sector:
#     for j in total_output_sector[i]:
#         total_output_sector['ROW'][j] += total_output_sector[i][j]

In [52]:
# dominator = dictsum(total_output_sector['ROW'])
# for i in A:
#     total_output_sector['ROW'][i] = (total_output_sector['ROW'][i]/dominator) * total_output['ROW']

In [53]:
# # check correctness
# dictsum(total_output_sector['ROW']),total_output['ROW']

In [54]:
file=open("total_output_sector.json",'w')
json.dump(total_output_sector,file)
file.close()